In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model

def detect_dizziness():
    # Load the pre-trained face detector
    face_cascade_path = 'haarcascade_frontalface_default.xml'
    if not os.path.exists(face_cascade_path):
        print("this line")
        return
    face_cascade = cv2.CascadeClassifier(face_cascade_path)

    #Load the pre-trained dizziness detection model
    model_path = 'dizziness_detection_model.h5'
    if not os.path.exists(model_path):
        print("This line")
        return
    dizziness_model = load_model(model_path)

    # Open the video capture device
    cap = cv2.VideoCapture(0)

    while True:
        # Read a frame from the video capture device
        ret, frame = cap.read()

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # For each face, predict dizziness
        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            face_roi = cv2.resize(face_roi, (96, 96))
            face_roi = face_roi / 255.0
            face_roi = np.expand_dims(face_roi, axis=0)
            face_roi = np.expand_dims(face_roi, axis=-1)
            dizziness = dizziness_model.predict(face_roi)[0][0]

            # If the driver is dizzy, print a message and break out of the loop
            if dizziness == 1:
                print("Driver is dizzy!")
                break

            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("Driver Dizziness Detector", frame)

        # Wait for a key press
        if cv2.waitKey(1) == ord('q'):
            break

    # Release the video capture device and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Call the detect_dizziness function to start detecting dizziness
detect_dizziness()



this line


In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras

from glob import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split


X = []
Y = []

for i in tqdm(glob("/content/WhatsApp Image 2023-10-08 at 9.48.58 PM (1).jpeg")):
    temp = np.array(Image.open(i).resize((64,64)))
    X.append(temp)
    Y.append(1)

for i in tqdm(glob("/content/WhatsApp Image 2023-10-08 at 9.49.28 PM (1).jpeg")):
    temp = np.array(Image.open(i).resize((64,64)))
    X.append(temp)
    Y.append(0)

X = (np.array(X) - np.min(X)) / (np.max(X) - np.min(X))
Y = (np.array(Y) - np.min(Y)) / (np.max(Y) - np.min(Y))

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

datagen.fit(X_train)

history = model.fit(datagen.flow(X_train, Y_train, batch_size=32),
                    epochs=50,
                    validation_data=(X_test, Y_test))

test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)
model.save('dizziness_detection_model.h5')

100%|██████████| 1/1 [00:00<00:00, 963.10it/s]

Epoch 1/50


1/1 [==============================] - 1s 1s/step - loss: 0.6925 - accuracy: 1.0000 - val_loss: 0.8114 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 62ms/step - loss: 0.5817 - accuracy: 1.0000 - val_loss: 1.0077 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 76ms/step - loss: 0.3976 - accuracy: 1.0000 - val_loss: 1.4493 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 63ms/step - loss: 0.5138 - accuracy: 1.0000 - val_loss: 2.1591 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 85ms/step - loss: 0.0697 - accuracy: 1.0000 - val_loss: 3.3330 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 76ms/step - loss: 0.0768 - accuracy: 1.0000 - val_loss: 5.1522 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 76ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 7.3515 - val_accuracy: 0.0000e+00
Epoch 8/50
1

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
